In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
import os
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
epochs= 5
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

dense_layers = [0]
layer_sizes = [64]
conv_layers = [1]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer,
                                                         int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs\{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )
            log_dir= os.path.join('logs','fit','{}'.format(NAME))
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1 ,profile_batch = 100000000 )

            history=model.fit(X, y,
                      batch_size=32,
                      epochs=2,
                      validation_split=0.3,
                      callbacks=[tensorboard_callback])
            loss = history.history['loss']
            for epoch in range(epochs):
          # Reset the metrics at the start of the next epoch
                  train_loss.reset_states()
                  #train_accuracy.reset_states()
                  template = 'Epoch {}, Loss: {}'
                  print(template.format(epoch + 1,train_loss.result()))
    model.save('64x3-CNN.model') 


    
print()

1-conv-64-nodes-0-dense-1589508257
Train on 445 samples, validate on 191 samples
Epoch 1/2
445/445 [==============================] - 6s 14ms/sample - loss: 0.9853 - accuracy: 0.7416 - val_loss: 0.3022 - val_accuracy: 0.9215
Epoch 2/2
445/445 [==============================] - 6s 13ms/sample - loss: 0.1605 - accuracy: 0.9438 - val_loss: 0.1462 - val_accuracy: 0.9529
Epoch 1, Loss: 0.0
Epoch 2, Loss: 0.0
Epoch 3, Loss: 0.0
Epoch 4, Loss: 0.0
Epoch 5, Loss: 0.0
INFO:tensorflow:Assets written to: 64x3-CNN.model\assets



In [63]:
import matplotlib.pyplot as plt
from matplotlib import style

style.use("ggplot")

model_name = "hamada.model" #"model-1570499409" # grab whichever model name you want here. We could also just reference the MODEL_NAME if you're in a notebook still.


def create_acc_loss_graph(model_name):
    contents = open("//DESKTOP-HU23BKF/Users/20115/64x3-CNN.model", "r").read().split("\n")

    times = []
    accuracies = []
    losses = []

    val_accs = []
    val_losses = []

    for c in contents:
        if model_name in c:
            name, timestamp, acc, loss, val_acc, val_loss = c.split(",")

            times.append(float(timestamp))
            accuracies.append(float(acc))
            losses.append(float(loss))

            val_accs.append(float(val_acc))
            val_losses.append(float(val_loss))


    fig = plt.figure()

    ax1 = plt.subplot2grid((2,1), (0,0))
    ax2 = plt.subplot2grid((2,1), (1,0), sharex=ax1)


    ax1.plot(times, accuracies, label="acc")
    ax1.plot(times, val_accs, label="val_acc")
    ax1.legend(loc=2)
    ax2.plot(times,losses, label="loss")
    ax2.plot(times,val_losses, label="val_loss")
    ax2.legend(loc=2)
    plt.show()

create_acc_loss_graph(model_name)


PermissionError: [Errno 13] Permission denied: '//DESKTOP-HU23BKF/Users/20115/64x3-CNN.model'

In [3]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

NameError: name 'EPOCHS' is not defined

In [41]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
import os
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
epochs= 5
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

model = createmodel()
@tf.function
def train_step(X, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(X, training=True)
    print(predictions)
    loss = loss_object(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

for epoch in range(epochs):
              # Reset the metrics at the start of the next epoch
              train_loss.reset_states()
              train_accuracy.reset_states()
              template = 'Epoch {}, Loss: {}, Accuracy: {}'
              print(template.format(epoch + 1,train_loss.result(),train_accuracy.result() * 100))


1-conv-64-nodes-0-dense-1589507660
Train on 445 samples, validate on 191 samples
Epoch 1/2
445/445 [==============================] - 6s 14ms/sample - loss: 2.3076 - accuracy: 0.7169 - val_loss: 0.9206 - val_accuracy: 0.8482
Epoch 2/2
445/445 [==============================] - 6s 13ms/sample - loss: 0.5053 - accuracy: 0.8764 - val_loss: 0.2573 - val_accuracy: 0.9372
INFO:tensorflow:Assets written to: 64x3-CNN.model\assets
Epoch 1, Loss: 0.0, Accuracy: 0.0
Epoch 2, Loss: 0.0, Accuracy: 0.0
Epoch 3, Loss: 0.0, Accuracy: 0.0
Epoch 4, Loss: 0.0, Accuracy: 0.0
Epoch 5, Loss: 0.0, Accuracy: 0.0
